In [12]:
import pandas as pd
import numpy as np
import datetime as dt
from pandas_datareader import data as pdr

def getData(stocks, start, end):
    stockData = pdr.get_data_yahoo(stocks, start=start, end=end)
    stockData = stockData['Close']
    returns= stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return returns, meanReturns, covMatrix

def portfolioPerformance(weights, meanReturns, covMatrix, Time):
    returns = np.sum(meanReturns*weights)*Time
    std = np.sqrt(np.dot(weights.T, np.dot(covMatrix, weights)))*np.sqrt(Time)
    return returns, std

stocklist = ['CBA', 'BHP', 'TLS', 'NAB', 'WBC', 'STO']
stocks = [stock+'.AX' for stock in stocklist]
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days= 800)

returns, meanReturns, covMatrix = getData(stocks, start=startDate, end=endDate)
returns = returns.dropna()

weights = np.random.random(len(returns.columns))
weights /= np.sum(weights)

returns['portfolio'] = returns.dot(weights)
#print(returns)

def historicalVaR(returns, alpha=5):     #alpha confidence interval
    ''' read in a pnadas dataframe of returns / a pandas series of returns'''
    ''' output the percentile of the dist at the given alpha conf interval'''
    if isinstance(returns, pd.Series):
        return np.percentile(returns, alpha)
    elif isinstance(returns, pd.DataFrame):
        return returns.agg(historicalVaR, alpha=5)
    else:
        raise TypeError("expected returns to be dataframe or series")
        

    
def historicalCVaR(returns, alpha=5):
    if isinstance(returns, pd.Series):
        belowVaR = returns <= historicalVaR(returns, alpha=alpha)
        return returns[belowVaR].mean()
    elif isinstance(returns, pd.DataFrame):
        return returns.agg(historicalCVaR, alpha=5)
    else:
        raise TypeError("expected returns to be dataframe or series")


Time = 100

VaR = -historicalVaR(returns['portfolio'], alpha=5)*np.sqrt(Time)
CVaR = -historicalCVaR(returns['portfolio'], alpha=5)*np.sqrt(Time)
pReturn, pStd = portfolioPerformance(weights, meanReturns, covMatrix, Time)

InitialInvestment = 100000
print("Expected Portfolio Return:       ", round(InitialInvestment*pReturn,2))
print("Value at Risk 95th CI:       ", round(InitialInvestment*VaR,2))
print("Conditional Value at Risk 95th CI:       ", round(InitialInvestment*CVaR,2))




Expected Portfolio Return:        8738.25
Value at Risk 95th CI:        23081.86
Conditional Value at Risk 95th CI:        37615.51
